In [ ]:
!pip install transformers datasets peft numpy
!pip install --upgrade datasets
!pip install tabulate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 36.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [3]:
import torch
from transformers import EarlyStoppingCallback, AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, TrainerCallback
from peft import get_peft_model, LoraConfig, TaskType
from datasets import load_dataset
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import set_seed
from tabulate import tabulate
from itertools import product

set_seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

dataset = load_dataset("ag_news")

def tokenize(batch, tokenizer):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=128)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="macro")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

class AccuracyThresholdStoppingCallback(TrainerCallback):
    def __init__(self, target_accuracy=0.85):
        self.target_accuracy = target_accuracy
        self.converged_step = None

    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        if metrics and "eval_accuracy" in metrics:
            acc = metrics["eval_accuracy"]
            print(f"Step {state.global_step}: eval_accuracy={acc:.4f}")
            if acc >= self.target_accuracy:
                print(f"Target accuracy {self.target_accuracy} reached at step {state.global_step}. Stopping training.")
                self.converged_step = state.global_step
                control.should_training_stop = True
        return control

def train_eval_lora(model_name, r, lora_alpha, lora_dropout, target_acc=0.85):
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    full_train = dataset["train"].map(lambda x: tokenize(x, tokenizer), batched=True)
    full_train.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

    full_test = dataset["test"].map(lambda x: tokenize(x, tokenizer), batched=True)
    full_test.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

    base_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=4).to(device)

    peft_config = LoraConfig(
        task_type=TaskType.SEQ_CLS,
        r=r,
        lora_alpha=lora_alpha,
        lora_dropout=lora_dropout,
        bias="none",
        target_modules=["query", "value"]
    )
    model = get_peft_model(base_model, peft_config).to(device)
    model.print_trainable_parameters()

    early_stopping_cb = AccuracyThresholdStoppingCallback(target_accuracy=target_acc)

    training_args = TrainingArguments(
        output_dir=f"./{model_name.replace('/', '_')}_lora_r{r}_alpha{lora_alpha}_drop{lora_dropout}",
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        max_steps=3000,
        num_train_epochs=1,
        logging_dir="./logs",
        logging_steps=50,
        save_total_limit=2,
        fp16=torch.cuda.is_available(),
        eval_strategy="steps",
        eval_steps=50,
        load_best_model_at_end=True,
        metric_for_best_model="eval_accuracy",
        greater_is_better=True,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=full_train,
        eval_dataset=full_test,
        compute_metrics=compute_metrics,
        callbacks=[early_stopping_cb]
    )

    print(f"\nTraining {model_name} with LoRA params: r={r}, alpha={lora_alpha}, dropout={lora_dropout}")
    trainer.train()

    converged_step = early_stopping_cb.converged_step
    metrics = trainer.evaluate()
    print(f"Final Metrics: {metrics}")

    return metrics["eval_accuracy"], converged_step

def eval_base_model(model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    full_test = dataset["test"].map(lambda x: tokenize(x, tokenizer), batched=True)
    full_test.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

    base_model_plain = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=4).to(device)

    base_trainer = Trainer(
        model=base_model_plain,
        args=TrainingArguments(output_dir=f"./base_eval_{model_name.replace('/', '_')}", per_device_eval_batch_size=32, fp16=torch.cuda.is_available()),
        eval_dataset=full_test,
        compute_metrics=compute_metrics,
    )
    base_metrics = base_trainer.evaluate()
    return base_metrics["eval_accuracy"]

model_names = ["google/bert_uncased_L-2_H-128_A-2", "prajjwal1/bert-tiny"]

results = []

# Evaluate base models
for model_name in model_names:
    print(f"\nEvaluating base (no LoRA) model: {model_name}")
    base_acc = eval_base_model(model_name)
    print(f"Base accuracy for {model_name}: {base_acc:.4f}")
    results.append({
        "model": model_name,
        "r": None,
        "alpha": None,
        "dropout": None,
        "accuracy": base_acc,
        "converged_step": None,
        "note": "base"
    })

# Grid of r and alpha
r_values = [8, 16, 32]
alpha_values = [256, 512]

# Evaluate all combinations
for model_name in model_names:
    for r, alpha in product(r_values, alpha_values):
        acc, step = train_eval_lora(model_name, r=r, lora_alpha=alpha, lora_dropout=0.0, target_acc=0.84)
        results.append({
            "model": model_name,
            "r": r,
            "alpha": alpha,
            "dropout": 0.0,
            "accuracy": acc,
            "converged_step": step,
            "note": "lora"
        })



Using device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


trainable params: 33,284 || all params: 4,419,720 || trainable%: 0.7531

Training prajjwal1/bert-tiny with LoRA params: r=32, alpha=512, dropout=0.0


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
50,1.357100,1.309956,0.550263,0.559942,0.550263,0.539097
100,1.273900,1.199981,0.706974,0.708744,0.706974,0.707791
150,1.158700,1.067270,0.771184,0.768389,0.771184,0.768822
200,1.039400,0.948010,0.801053,0.803034,0.801053,0.799410
250,0.934700,0.848394,0.815395,0.814790,0.815395,0.815008
300,0.836900,0.771765,0.817105,0.815975,0.817105,0.814811
350,0.763800,0.715702,0.826842,0.827029,0.826842,0.826503
400,0.693500,0.663977,0.833158,0.832589,0.833158,0.831941
450,0.667700,0.626410,0.842500,0.842872,0.842500,0.842573


Step 50: eval_accuracy=0.5503
Step 100: eval_accuracy=0.7070
Step 150: eval_accuracy=0.7712
Step 200: eval_accuracy=0.8011
Step 250: eval_accuracy=0.8154
Step 300: eval_accuracy=0.8171
Step 350: eval_accuracy=0.8268
Step 400: eval_accuracy=0.8332
Step 450: eval_accuracy=0.8425
Target accuracy 0.84 reached at step 450. Stopping training.


Step 450: eval_accuracy=0.8425
Target accuracy 0.84 reached at step 450. Stopping training.
Final Metrics: {'eval_loss': 0.6264098286628723, 'eval_accuracy': 0.8425, 'eval_precision': 0.8428723546740378, 'eval_recall': 0.8424999999999999, 'eval_f1': 0.8425732040448486, 'eval_runtime': 1.4684, 'eval_samples_per_second': 5175.854, 'eval_steps_per_second': 162.086, 'epoch': 0.12}


In [4]:

# Final results
print("\n=== Final Results ===")
print(tabulate(results, headers="keys", floatfmt=".4f"))


=== Final Results ===
model                                r    alpha    dropout    accuracy    converged_step  note
---------------------------------  ---  -------  ---------  ----------  ----------------  ------
google/bert_uncased_L-2_H-128_A-2                               0.1766                    base
prajjwal1/bert-tiny                                             0.2478                    base
google/bert_uncased_L-2_H-128_A-2    8      256     0.0000      0.8363              1100  lora
google/bert_uncased_L-2_H-128_A-2    8      512     0.0000      0.8183               700  lora
google/bert_uncased_L-2_H-128_A-2   16      256     0.0000      0.8363               700  lora
google/bert_uncased_L-2_H-128_A-2   16      512     0.0000      0.8420               450  lora
google/bert_uncased_L-2_H-128_A-2   32      256     0.0000      0.8442               500  lora
google/bert_uncased_L-2_H-128_A-2   32      512     0.0000      0.8474               500  lora
prajjwal1/bert-tiny      